In [3]:
import os
import sys

os.environ["JAVA_HOME"] = "JDK 8"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
# from pyspark.sql.types import IntegerType

# from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Saving and Project").getOrCreate()


In [50]:
data = [(1, "Moby Dick", "Herman Melville", 1851),
        (2, "Treasure Island", "Robert Louis Stevenson", 1883),
        (3, "Robinson Crusoe", "Daniel Defoe", 1719),
        (4, "The Fellowship of the Ring", "J.R.R. Tolkien", 1954),
        (5, "The Hitchhiker's Guide to the Galaxy", "Douglas Adams", 1979)]

columns = ["ID", "book_name", "author", "publish_date"]

In [51]:
df_books = spark.createDataFrame(data, columns)
df_books.show()


+---+--------------------+--------------------+------------+
| ID|           book_name|              author|publish_date|
+---+--------------------+--------------------+------------+
|  1|           Moby Dick|     Herman Melville|        1851|
|  2|     Treasure Island|Robert Louis Stev...|        1883|
|  3|     Robinson Crusoe|        Daniel Defoe|        1719|
|  4|The Fellowship of...|      J.R.R. Tolkien|        1954|
|  5|The Hitchhiker's ...|       Douglas Adams|        1979|
+---+--------------------+--------------------+------------+



In [10]:
output_path = "../"
df_books.write.csv(output_path, header=True, mode="overwrite")

25/02/01 10:21:12 ERROR Executor: Exception in task 2.0 in stage 4.0 (TID 14) 4]
java.lang.InternalError: java.io.FileNotFoundException: /home/yuri/Documents/Data Engineering/PySpark/JDK 8/jre/lib/ext/cldrdata.jar
	at sun.misc.URLClassPath$JarLoader.getResource(URLClassPath.java:1066)
	at sun.misc.URLClassPath.getResource(URLClassPath.java:250)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:366)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:363)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:362)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at java.util.ResourceBundle$RBClassLoader.loadClass(ResourceBundle.java:512)
	at java.util.ResourceBundle$Control.newBundle(ResourceBundle.java:2657)
	at java.util.ResourceBundle.loadBundle(ResourceBundle.java:1518)
	at java.util.ResourceBundle.findBundle(ResourceBundle.java:1482)
	at java.u

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58144)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/home/yuri/Documents/Data Engineering/PySpark/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
  File "/home/yuri/Documents/Data Engineering/PySpark/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 267, in poll
  File "/home/yuri/Documents/Data Engineering/PySpark/.venv/lib/python3.11/site-p

Py4JError: py4j does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/yuri/Documents/Data Engineering/PySpark/.venv/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yuri/Documents/Data Engineering/PySpark/.venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
  File "/home/yuri/Documents/Data Engineering/PySpark/.venv/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Once you have this downloaded, load it into your Python file and into a PySpark DataFrame. Firstly check the file to ensure that the data is of the right type, and there are no null values. Then I want you to tell me which day had the most lightning strikes recorded. Order the dataframe by descending number of strikes, then save it to a CSV file.

In [10]:
df_lightning = spark.read.csv("lightening strikes dataset.csv", header=True, inferSchema=True)

In [14]:
df_lightning.show(5)

+----------+-----------------+-----------------+
|      date|number_of_strikes|center_point_geom|
+----------+-----------------+-----------------+
|2018-01-03|              194|    POINT(-75 27)|
|2018-01-03|               41|  POINT(-78.4 29)|
|2018-01-03|               33|  POINT(-73.9 27)|
|2018-01-03|               38|  POINT(-73.8 27)|
|2018-01-03|               92|    POINT(-79 28)|
+----------+-----------------+-----------------+
only showing top 5 rows



In [28]:
df_bydays = df_lightning.groupBy("date").agg(sum("number_of_strikes").alias("total_count")).orderBy(col("total_count").desc())


In [34]:
output_path = "csv_out"
df_bydays.write.csv(output_path, header=True)

In [45]:
df_lightning.groupBy("center_point_geom").agg(round(avg("number_of_strikes"), 2).alias("avg_strikes")).show()

+-----------------+-----------+
|center_point_geom|avg_strikes|
+-----------------+-----------+
|POINT(-95.3 29.4)|      17.52|
|POINT(-92.6 24.9)|       6.65|
|POINT(-92.2 26.1)|       8.64|
|POINT(-91.3 28.1)|      11.44|
|POINT(-93.9 28.1)|       12.3|
|POINT(-87.4 25.2)|       8.94|
|POINT(-86.9 26.4)|      11.91|
|POINT(-76.3 26.7)|      14.53|
|  POINT(-85.5 26)|      10.03|
|POINT(-76.1 25.7)|       11.0|
|POINT(-99.9 33.6)|      26.79|
|  POINT(-77 26.5)|       9.83|
|POINT(-79.6 28.3)|      16.95|
|POINT(-78.7 31.9)|      16.14|
|POINT(-73.5 29.2)|       5.57|
|POINT(-96.5 27.7)|      19.76|
|POINT(-95.9 33.7)|      27.16|
|POINT(-95.1 34.3)|       34.9|
|POINT(-94.7 35.2)|      39.15|
|  POINT(-92.8 31)|      25.78|
+-----------------+-----------+
only showing top 20 rows



In [52]:
import pandas as pd
pandas_df = df_books.toPandas()
pandas_df.to_csv("book.csv")